In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions
from bs4 import BeautifulSoup
import re

In [39]:
url = "https://www.moneycontrol.com/news/business/stocks/buy-home-first-finance-target-of-rs-1250-motilal-oswal-12898963.html"

In [40]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")


In [41]:
driver = webdriver.Chrome(service=service, options= options)

In [42]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 30).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()
            
            if len(para_text) < 50 :
                continue
            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue
            
            paragraphs_list.append(para_text)
            
            
    news.update({"content": paragraphs_list})
            
    driver.quit()   
    return news

In [43]:
news = extract_data(url)

In [44]:
news

{'title': 'Buy Home First Finance; target of Rs 1250: Motilal Oswal',
 'desc': 'Motilal Oswal is bullish on Home First Finance recommended buy rating on the stock with a target price of Rs 1250 in its research report dated December 26, 2024.',
 'date': 'December 27, 2024',
 'datetime': 'December 27, 2024 / 15:12 IST',
 'content': ["Motilal Oswal's research report on Home First Finance",
  'We met with the senior management of Home First Finance, represented by Mr. Manoj Viswanathan, MD & CEO, and Ms. Nutan Gaba Patwari, CFO, to gain insights into the current industry dynamics in affordable housing finance and the way forward for the company. Following are the key takeaways: 1) Home First is not seeing any spillover of stress from microfinance or short-term personal loans (STPL), and its asset quality remains largely stable; 2) demand continues to remain strong in the affordable segment, particularly in self-construction; 3) the ability to penetrate deeper into its existing states and i